# Imports

In [ ]:
!cp /content/drive/MyDrive/chronos-forecasting-main.zip /content

In [ ]:
%pip install '/content/chronos-forecasting-main.zip' 'pandas[pyarrow]' 'matplotlib' 'gluonts' 'mxnet' #'chronos-forecasting>=2.1'

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Processing ./chronos-forecasting-main.zip
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for chronos-forecasting: filename=chronos_forecasting-2.2.0rc2-py3-none-any.whl size=72110 sha256=d133ebc2ec448e707fa24a92379da9147b2b79a1c2d5293ca26f5287e7cfe106
  Stored in directory: /root/.cache/pip/wheels/90/f2/a5/1a9fa43245bfa86e7db3996abc8745b922dac4fcbe84ff3d44
Successfully built chronos-forecasting
  Attempting uninstall: chronos-forecasting
    Found existing installation: chronos-forecasting 2.2.0rc2
    Uninstalling chronos-forecasting-2.2.0rc2:
      Successfully uninstalled chronos-forecasting-2.2.0rc2


In [ ]:

# Use only 1 GPU if available
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


from chronos import BaseChronosPipeline, Chronos2Pipeline

# Load the Chronos-2 pipeline
# GPU recommended for faster inference, but CPU is also supported
pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained("amazon/chronos-2", device_map="cuda")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/478M [00:00<?, ?B/s]

# Test runs of chronos zero shot

In [ ]:
# Load data as a long-format pandas data frame
context_df = pd.read_csv("https://autogluon.s3.amazonaws.com/datasets/timeseries/m4_hourly/train.csv")
print("Input dataframe shape:", context_df.shape)
display(context_df.head())

Input dataframe shape: (353500, 3)


,item_id,timestamp,target
0,H1,1750-01-01 00:00:00,605.0
1,H1,1750-01-01 01:00:00,586.0
2,H1,1750-01-01 02:00:00,586.0
3,H1,1750-01-01 03:00:00,559.0
4,H1,1750-01-01 04:00:00,511.0


In [ ]:
pred_df = pipeline.predict_df(context_df, prediction_length=24, quantile_levels=[0.1, 0.5, 0.9])

print("Output dataframe shape:", pred_df.shape)
display(pred_df.head())

Output dataframe shape: (9936, 7)


,item_id,timestamp,target_name,predictions,0.1,0.5,0.9
0,H1,1750-01-30 04:00:00,target,624.867981,611.385132,624.867981,638.598755
1,H1,1750-01-30 05:00:00,target,563.703125,546.655090,563.703125,578.665649
2,H1,1750-01-30 06:00:00,target,521.589905,505.747498,521.589905,537.950806
3,H1,1750-01-30 07:00:00,target,489.910706,473.671814,489.910706,508.854187
4,H1,1750-01-30 08:00:00,target,471.144501,452.199463,471.144501,491.050293


In [ ]:
import pickle

with open('/content/drive/MyDrive/spy5m_smallta_fracdiff.pkl', 'rb') as f:
        context_df = pickle.load(f)
context_df = context_df[:20_000][['close']].reset_index().rename(columns={'close': 'target'})
# attempt to clean or simplify datatimes
# context_df['timestamp'] = pd.to_datetime(context_df['timestamp'], utc=True)
# context_df['timestamp'] = context_df['timestamp'].dt.tz_localize(None)
# but the gaps dont allow for inferring frequency so reset to false times without missing
context_df['timestamp'] = pd.date_range(start='2003-10-17 19:15:00', freq="5min", periods=20_000)
context_df['item_id'] = 'SP500'
pl = 78

In [ ]:

context_df

,timestamp,target,item_id
0,2003-10-17 19:15:00,17.502863,SP500
1,2003-10-17 19:20:00,17.597682,SP500
2,2003-10-17 19:25:00,17.539202,SP500
3,2003-10-17 19:30:00,17.471920,SP500
4,2003-10-17 19:35:00,17.547311,SP500
...,...,...,...
19995,2003-12-26 05:30:00,19.303079,SP500
19996,2003-12-26 05:35:00,19.293100,SP500
19997,2003-12-26 05:40:00,19.340610,SP500
19998,2003-12-26 05:45:00,19.336112,SP500


In [ ]:
pred_df = pipeline.predict_df(context_df, prediction_length=pl, quantile_levels=[0.1, 0.5, 0.9])

print("Output dataframe shape:", pred_df.shape)
display(pred_df.head())

Output dataframe shape: (78, 7)


,item_id,timestamp,target_name,predictions,0.1,0.5,0.9
0,SP500,2003-12-26 05:55:00,target,19.328047,19.283157,19.328047,19.365433
1,SP500,2003-12-26 06:00:00,target,19.330223,19.270338,19.330223,19.379997
2,SP500,2003-12-26 06:05:00,target,19.330540,19.259100,19.330540,19.393562
3,SP500,2003-12-26 06:10:00,target,19.331768,19.252707,19.331768,19.402378
4,SP500,2003-12-26 06:15:00,target,19.328890,19.244593,19.328890,19.416950


In [ ]:
pred_df.to_csv('pred_example.csv')

In [ ]:
pred_df = pd.read_csv('pred_example.csv')

In [ ]:
pred_df[['0.1','0.5','0.9']].to_numpy().shape

(78, 3)

# Eval

In [ ]:
from gluonts.time_feature import norm_freq_str

DEFAULT_SEASONALITIES = {
    "S": 3600,  # 1 hour
    "s": 3600,  # 1 hour
    "T": 1440,  # 1 day
    "min": 1440,  # 1 day
    "H": 24,  # 1 day
    "h": 24,  # 1 day
    "D": 1,  # 1 day
    "W": 1,  # 1 week
    "M": 12,
    "ME": 12,
    "B": 5,
    "Q": 4,
    "QE": 4,
}

def get_seasonality(freq: str, seasonalities=DEFAULT_SEASONALITIES) -> int:
    """
    Return the seasonality of a given frequency:

    >>> get_seasonality("2h")
    12
    """
    offset = pd.tseries.frequencies.to_offset(freq)

    base_seasonality = seasonalities.get(norm_freq_str(offset.name), 1)

    seasonality, remainder = divmod(base_seasonality, offset.n)
    if not remainder:
        return seasonality

    logger.warning(
        f"Multiple {offset.n} does not divide base seasonality "
        f"{base_seasonality}. Falling back to seasonality 1."
    )

    return 1

In [ ]:
from sklearn.metrics import mean_absolute_error

def eval_mase(y, y_hat, pl, seasonality):
  '''
  y:  np.array, original series of testset including forecasted values
  yhat: np.array, forecasted values
  pl: int, prediction length
  seasonality: int, seasonality of the time series
  '''
  training_data = y[:-pl]
  ground_truth = y[-pl:]

  y_pred_naive = np.array(training_data)[:-int(seasonality)]
  mae_naive = mean_absolute_error(np.array(training_data)[int(seasonality):], y_pred_naive, multioutput="uniform_average")

  mae_score = mean_absolute_error(
      ground_truth,
      y_hat,
      sample_weight=None,
      multioutput="uniform_average",
  )

  epsilon = np.finfo(np.float64).eps

  mase_score = mae_score / np.maximum(mae_naive, epsilon)
  return mase_score

In [ ]:
get_seasonality('5min') # gluonts default seasonality for mase

288

In [ ]:
def get_weights_ffd(diff_amt, thresh, lim):
    weights = [1.]
    k = 1
    ctr = 0
    while True:
        # compute the next weight
        weights_ = -weights[-1] * (diff_amt - k + 1) / k
        if abs(weights_) < thresh:
            break
        weights.append(weights_)
        k += 1
        ctr += 1
        if ctr == lim - 1:  # if we have reached the size limit, exit the loop
            break
    weights = np.array(weights[::-1]).reshape(-1, 1)
    return weights

def invert_ffd(new_fd_series, original_series, d, thresh, full_original=False):
    # reminder that original_series means training_series without ground truth of pred, although it should overwrite
    original_series.reset_index(drop=True, inplace=True) # reset index to ensure regular integer index for easier slicing
    weights = get_weights_ffd(d, thresh, len(original_series))
    width = len(weights) - 1 # -1 to allow for slicing a window of len(weights)
    new_unfd_series = original_series.copy()
    new_fd_series = new_fd_series.copy()
    if full_original:
        new_fd_series.index += width # shift the index to align with the original series index, i.e. before nans were dropped and the index reset on the frac diffed series

    for i in range(original_series.index[-1]+1, new_fd_series.index[-1]+1): # weirdly taking the last index + 1 instead of len/shape because of the index shift on the new fd series
        new_unfd_series.loc[i] = (new_fd_series.loc[i] - np.dot(weights[:-1].T, new_unfd_series[i-width:i]))[0]
    return new_unfd_series

In [ ]:

def pred_eval(context_df, pipeline, pl, quantile_levels=[0.1, 0.5, 0.9], seasonality=288, d=None, thresh=None):

  def invert_pred(pred_df, context_df, d, thresh):
    training_context = context_df[:-pl]
    full_fd_series_pred = pd.concat([training_context[['target']], pred_df[['predictions']]])
    full_unfd_series = invert_ffd(full_fd_series_pred, training_context['target_o'], d, thresh)
    return full_unfd_series[-pl:]

  # input df, default target col str is "target" other columns are taken as past covariates
  context_df.reset_index(inplace=True, drop=True)
  # doesnt seem to handle missing easily, so replace with incorrect dates with correct frequency
  context_df['timestamp'] = pd.date_range(start='2003-10-17 19:15:00', freq="5min", periods=len(context_df))
  context_df['item_id'] = 'SP500'
  pred_df = pipeline.predict_df(context_df, prediction_length=pl, quantile_levels=quantile_levels)

  y_hat = pred_df[['predictions']].to_numpy()
  y = context_df[['target']].to_numpy()

  if d is not None and thresh is not None:
    y_hat = invert_pred(pred_df, context_df, d, thresh)
    y = context_df[['target_o']].to_numpy()

  mase = eval_mase(y, y_hat, pl=pl, seasonality=seasonality)
  return mase, pred_df


In [ ]:
import pickle

with open('/content/drive/MyDrive/spy5m_bintp_labelled.pkl', 'rb') as f:
    df_original = pickle.load(f) # ohlv + transactions + labels + bintp labels

with open('/content/drive/MyDrive/spy5m_fd.pkl', 'rb') as f:
    df_fd = pickle.load(f) # d=0.25 when using this one, post h=l drop
# with open('/content/drive/MyDrive/spy5m_bintp004_episodes_fracdiff.pkl', 'rb') as f:
#     df_fd = pickle.load(f) # fracdiffed ohlcv + transactions + labels
# with open('/mnt/c/Users/resha/Documents/Github/balancing_framework/spy5m_labelled_episodes_ta.pkl', 'rb') as f:
#     df_ta = pickle.load(f) # ohlcv + ~120 TA features + labels
# with open('/mnt/c/Users/resha/Documents/Github/balancing_framework/spy5m_ta_fracdiff.pkl', 'rb') as f:
#     df_fd_ta = pickle.load(f) # fracdiffed ohlcv + transactions + ~120 TA features + labels
# with open('spy5m_smallta.pkl', 'rb') as f:
#     df_ta = pickle.load(f)
# with open('spy5m_smallta_fracdiff.pkl', 'rb') as f:
#     df_fd = pickle.load(f)

# filter single order bars
df_original.drop(df_original[df_original['high'] == df_original['low']].index, inplace=True)
# df_fd.drop(df_fd[df_fd['high'] == df_fd['low']].index, inplace=True)

# df_original.drop(df_original[df_original['transactions'] == 1].index, inplace=True)
# df_fd.drop(df_fd[df_fd['transactions'] == 1].index, inplace=True)


d = 0.25 # 0.2 0.25
thresh = 1e-5

df_size = 20_000
# original series target with ohlcv
df = df_original[["volume", "vwap", "open", "close", "high", "low", "transactions"]] # 0.01 0.001
end_ts = df_original[:df_size].index[-1]
X_withoutfd= df.loc[:end_ts].copy()
X_withoutfd['target'] = X_withoutfd['close'].shift(-1)
X_withoutfd.dropna(inplace=True)

# fd target with fd cols and original cols
df2 = df_fd[["volume", "vwap", "open", "close", "high", "low", "transactions"]]
X_fdtarget = df2.loc[:end_ts].copy()
X_fdtarget = X_fdtarget.join(X_withoutfd.add_suffix(f'_o'), how='outer')
X_fdtarget['target'] = X_fdtarget['close'].shift(-1)
X_fdtarget.dropna(inplace=True)

# switch above to have original target with fd in supp
X_otarget_fdinfdr = X_fdtarget.copy()
X_otarget_fdinfdr.rename(columns={"target": "target_fd", "target_o": "target"}, inplace=True)

del df2, df, df_fd, df_original


In [ ]:
pl=78
results = []
dataset = 'SP500'

# mase, pred_df = pred_eval(X_withoutfd, pipeline, pl)
# form = 'original'
# print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
# results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

# mase, pred_df = pred_eval(X_otarget_fdinfdr, pipeline, pl)
# form = 'original fd supp'
# print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
# results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

# mase, pred_df = pred_eval(X_fdtarget, pipeline, pl, d=d, thresh=thresh)
# form = 'fd'
# print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
# results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

# pl=12

# mase, pred_df = pred_eval(X_withoutfd, pipeline, pl)
# form = 'original'
# print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
# results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})


# mase, pred_df = pred_eval(X_otarget_fdinfdr, pipeline, pl)
# form = 'original fd supp'
# print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
# results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

# mase, pred_df = pred_eval(X_fdtarget, pipeline, pl, d=d, thresh=thresh)
# form = 'fd'
# print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
# results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

# for comp to finetuned
pl=1100

mase, pred_df = pred_eval(X_withoutfd, pipeline, pl)
form = 'original'
print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

mase, pred_df = pred_eval(X_otarget_fdinfdr, pipeline, pl)
form = 'original fd supp'
print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

mase, pred_df = pred_eval(X_fdtarget, pipeline, pl, d=d, thresh=thresh)
form = 'fd'
print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

/usr/local/lib/python3.12/dist-packages/chronos/chronos2/pipeline.py:572: UserWarning: We recommend keeping prediction length <= 1024. The quality of longer predictions may degrade since the model is not optimized for it. 
  warnings.warn(msg)


SP500 original pl 1100, mase: 1.6148


/usr/local/lib/python3.12/dist-packages/chronos/chronos2/pipeline.py:572: UserWarning: We recommend keeping prediction length <= 1024. The quality of longer predictions may degrade since the model is not optimized for it. 
  warnings.warn(msg)


SP500 original fd supp pl 1100, mase: 1.7275


/usr/local/lib/python3.12/dist-packages/chronos/chronos2/pipeline.py:572: UserWarning: We recommend keeping prediction length <= 1024. The quality of longer predictions may degrade since the model is not optimized for it. 
  warnings.warn(msg)


SP500 fd pl 1100, mase: 3.0995


In [ ]:
# dict to txt

import json

with open('sp500_zeroshot_results_2018.txt', 'w') as f:
    json.dump(results, f)

In [ ]:
import pickle

X = pd.read_csv('/content/drive/MyDrive/m4_1165_ffd.csv') # fixed width fd, 0.01 thresh, d=0.95
X.rename(columns={"values_o": "target_o", "values_fd": "target_fd"}, inplace=True)
dataset = 'm4_daily_dataset'
d=0.95
thresh = 0.01


X_withoutfd = X.drop(columns=['target_fd']).rename(columns={"target_o":"target"})
X_fdtarget = X.rename(columns={"target_fd": "target"})
# this X_fdtarget leaves o in fdr, fd_fdr is still the opposite to compare the impact of fd as target,
# and so far most tests have shown o+fd being more effective than fd alone
X_otarget_fdinfdr = X.rename(columns={"target_o": "target"})

In [ ]:
pl=14
results = []
dataset = 'm4_1165'

mase, pred_df = pred_eval(X_withoutfd, pipeline, pl)
form = 'original'
print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

mase, pred_df = pred_eval(X_otarget_fdinfdr, pipeline, pl)
form = 'original fd supp'
print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

mase, pred_df = pred_eval(X_fdtarget, pipeline, pl, d=0.95, thresh=0.01)
form = 'fd'
print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

pl=140

mase, pred_df = pred_eval(X_withoutfd, pipeline, pl)
form = 'original'
print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

mase, pred_df = pred_eval(X_otarget_fdinfdr, pipeline, pl)
form = 'original fd supp'
print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

mase, pred_df = pred_eval(X_fdtarget, pipeline, pl, d=0.95, thresh=0.01)
form = 'fd'
print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

# for comp to finetuned
pl=700

mase, pred_df = pred_eval(X_withoutfd, pipeline, pl)
form = 'original'
print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

mase, pred_df = pred_eval(X_otarget_fdinfdr, pipeline, pl)
form = 'original fd supp'
print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

mase, pred_df = pred_eval(X_fdtarget, pipeline, pl, d=0.95, thresh=0.01)
form = 'fd'
print(f'{dataset} {form} pl {pl}, mase: {mase:.4f}')
results.append({'dataset':dataset, 'form':form, 'pl':pl, 'mase':mase})

m4_1165 original pl 14, mase: 0.0996
m4_1165 original fd supp pl 14, mase: 0.1093
m4_1165 fd pl 14, mase: 0.1767
m4_1165 original pl 140, mase: 0.2997
m4_1165 original fd supp pl 140, mase: 0.3063
m4_1165 fd pl 140, mase: 0.2736
m4_1165 original pl 700, mase: 0.3524
m4_1165 original fd supp pl 700, mase: 0.3647
m4_1165 fd pl 700, mase: 2.5074


In [ ]:
# dict to txt

import json

with open('m4_1165_zeroshot_results.txt', 'w') as f:
    json.dump(results, f)

In [ ]:
# from upacking to inputs format before realizing predict df handles other cols
from tqdm import tqdm

df = X_fdtarget.copy()
target_col = 'close'
pl = 78

def prep_inputs(df, target_col):
  context_df = df.copy()
  context_df = context_df[[target_col]].reset_index().rename(columns={target_col: 'target'})
  covariates = context_df.columns.tolist()
  covariates.remove('target')
  # context_df['timestamp'] = pd.date_range(start='2003-10-17 19:15:00', freq="5min", periods=20_000)
  # context_df['item_id'] = 'SP500'
  inputs = [
      {
          "target": row[["target"]].values[0],
          "past_covariates": row[covariates].to_dict(),
      }
      for _, row in tqdm(context_df.iterrows())
  ]
  return inputs


In [ ]:
target_col = 'close'
pl = 78

inputs = prep_inputs(X_fdtarget, target_col)
quantiles, mean = pipeline.predict_quantiles(
    inputs, prediction_length=prediction_length, quantile_levels=[0.1, 0.5, 0.9]
)

KeyError: "None of [Index(['close'], dtype='object')] are in the [columns]"

In [ ]:
# wont enforce a weaker run than whats possible by using a shorter cl in finetuning or otherwise
# but should still note what was used

pipeline.model_context_length

In [ ]:

pred_df = pipeline.predict_df(context_df, prediction_length=pl, quantile_levels=[0.1, 0.5, 0.9])

y_hat = pred_df[['predictions']].to_numpy()
y = context_df[['target']].to_numpy()

mase = eval_mase(y, y_hat, pl=78, seasonality=288)


In [ ]:
# somewhere in the training the random seed gets set
# not sure how that follows through throughout exactly
# but trying to undo the set predictions seems like it would be a mess
# and then also cost more compute later in fine tuned comps
# so the snapshot should be fine i guess

import torch

seed = torch.initial_seed()

# Gluonts eval (not used)

In [ ]:
from gluonts.model.forecast import SampleForecast
from pandas._libs.tslibs.period import Period

forecasts = SampleForecast( , start_date=Period('2023-01-01', freq='D'), item_id='0')

In [ ]:
from gluonts.evaluation import Evaluator

def eval(y, yhat, quantiles=[0.1,0.5,0.9]):
  evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
  agg_metrics, item_metrics = evaluator(tss, forecasts)
  return agg_metrics

In [ ]:
from gluonts.dataset.repository import get_dataset, dataset_names
from gluonts.dataset.util import to_pandas
from gluonts.mx import SimpleFeedForwardEstimator, Trainer
from gluonts.evaluation import make_evaluation_predictions

dataset = get_dataset("m4_hourly")

estimator = SimpleFeedForwardEstimator(
    num_hidden_dimensions=[10],
    prediction_length=dataset.metadata.prediction_length,
    context_length=100,
    trainer=Trainer(ctx="cpu", epochs=1, learning_rate=1e-3, num_batches_per_epoch=100),
)

predictor = estimator.train(dataset.train)

forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.test,  # test dataset
    predictor=predictor,  # predictor
    num_samples=100,  # number of sample paths we want for evaluation
)

In [ ]:
f = list(forecast_it)
t = list(ts_it)

In [ ]:
f[0].samples.shape

In [ ]:
from gluonts.model.forecast import SampleForecast
from pandas._libs.tslibs.period import Period
import numpy as npa

SampleForecast( np.array([[1,2]]), start_date=Period('2023-01-01', freq='D'), item_id='0')

In [ ]:
dt = np.array(list(dataset.test))

In [ ]:
dt[0]['target'].shape